<a href="https://colab.research.google.com/github/jdavis8061/AAMU_AVC/blob/main/Coding_for_uav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello uav')

hello uav


In [2]:
import sys
print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [4]:
pip install mavsdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


In [5]:
#!/usr/bin/env python3

import asyncio
from mavsdk import System
from mavsdk.mission import MissionItem

async def run():
    # Initialize the drone
    drone = System()

    # Connect to the drone
    print("Connecting to drone...")
    await drone.connect(system_address="udp://:14540")

    # Wait for connection
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print("Drone connected!")
            break

    # Check if vehicle is ready
    print("Waiting for drone to have a global position estimate...")
    async for health in drone.telemetry.health():
        if health.is_global_position_ok:
            print("Global position estimate OK")
            break

    # Arm the drone
    print("Arming drone...")
    await drone.action.arm()

    # Take off
    print("Taking off...")
    await drone.action.takeoff()

    # Wait for drone to reach takeoff altitude
    await asyncio.sleep(10)

    # Define mission parameters
    mission_items = []
    mission_items.append(MissionItem(
        47.398039859999997,   # latitude_deg
        8.5455725400000002,   # longitude_deg
        25,                   # relative_altitude_m
        2,                    # speed_m_s
        True,                 # is_fly_through
        float('nan'),         # gimbal_pitch_deg
        float('nan'),         # gimbal_yaw_deg
        MissionItem.CameraAction.NONE,
        float('nan'),         # loiter_time_s
        float('nan'),         # camera_photo_interval_s
        1.0,                  # acceptance_radius_m
        0,                    # yaw_deg
        0                     # camera_photo_distance_m
    ))

    # Upload mission
    print("Uploading mission...")
    await drone.mission.upload_mission(mission_items)

    # Start mission
    print("Starting mission...")
    await drone.mission.start_mission()

    # Wait for mission to complete
    await asyncio.sleep(30)

    # Return to launch
    print("Returning to launch...")
    await drone.action.return_to_launch()

    # Wait for return to launch to complete
    await asyncio.sleep(20)

    # Land
    print("Landing...")
    await drone.action.land()

async def main():
    try:
        await run()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Run the asyncio loop
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [6]:
#!/usr/bin/env python3

import asyncio
import cv2
import numpy as np
from mavsdk import System
from mavsdk.mission import MissionItem
# Assume 'communicate_with_ugv' is a function to send data to the UGV
# Replace with your actual communication implementation
def communicate_with_ugv(marker_id, marker_position):
    """Communicates the detected marker information to the UGV."""
    print(f"Detected marker ID: {marker_id}, Position: {marker_position}")
    # Replace with your UGV communication logic
    # ...

async def run():
    # Initialize the drone
    drone = System()

    # Connect to the drone
    print("Connecting to drone...")
    await drone.connect(system_address="udp://:14540")

    # Wait for connection
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print("Drone connected!")
            break

    # Check if vehicle is ready
    print("Waiting for drone to have a global position estimate...")
    async for health in drone.telemetry.health():
        if health.is_global_position_ok:
            print("Global position estimate OK")
            break

    # Arm the drone
    print("Arming drone...")
    await drone.action.arm()

    # Take off
    print("Taking off...")
    await drone.action.takeoff()

    # Wait for drone to reach takeoff altitude
    await asyncio.sleep(10)

    # Define mission parameters - adjust for 30x30 field
    # Assume origin (0, 0) is at the center of the field
    field_size = 15  # Half the field size in feet (30/2)
    mission_items = []

    # Example mission waypoints - adjust for your desired coverage pattern
    mission_items.append(MissionItem(47.398039859999997 + (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 1
    mission_items.append(MissionItem(47.398039859999997 - (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 2
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 + (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 3
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 - (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 4


    # Upload mission
    print("Uploading mission...")
    await drone.mission.upload_mission(mission_items)

    # Start mission
    print("Starting mission...")
    await drone.mission.start_mission()

    # ArUco marker detection loop - run while mission is active
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    aruco_params = cv2.aruco.DetectorParameters_create()

    while True:
        # Get camera frame - replace with your camera feed
        # frame = get_camera_frame()
        # ...
        frame = np.zeros((480,640,3),np.uint8) # Replace with actual frame from camera

        # Detect ArUco markers
        corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)

        if ids is not None:
            for i in range(len(ids)):
                # Get marker ID and position
                marker_id = ids[i][0]
                marker_position = corners[i][0].mean(axis=0)  # Average corner coordinates for center

                # Communicate with UGV
                communicate_with_ugv(marker_id, marker_position)

        # Check mission status - break loop if mission is complete
        mission_progress = await drone.mission.mission_progress()
        if mission_progress.current == mission_progress.total:
            break

        # Add a delay to avoid excessive processing
        await asyncio.sleep(0.1)

    # Return to launch
    print("Returning to launch...")
    await drone.action.return_to_launch()

    # Wait for return to launch to complete
    await asyncio.sleep(20)

    # Land
    print("Landing...")
    await drone.action.land()

async def main():
    try:
        await run()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Run the asyncio loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())4#!/usr/bin/env python3

import asyncio
import cv2
import numpy as np
from mavsdk import System
from mavsdk.mission import MissionItem
# Assume 'communicate_with_ugv' is a function to send data to the UGV
# Replace with your actual communication implementation
def communicate_with_ugv(marker_id, marker_position):
    """Communicates the detected marker information to the UGV."""
    print(f"Detected marker ID: {marker_id}, Position: {marker_position}")
    # Replace with your UGV communication logic
    # ...

async def run():
    # Initialize the drone
    drone = System()

    # Connect to the drone
    print("Connecting to drone...")
    await drone.connect(system_address="udp://:14540")

    # Wait for connection
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print("Drone connected!")
            break

    # Check if vehicle is ready
    print("Waiting for drone to have a global position estimate...")
    async for health in drone.telemetry.health():
        if health.is_global_position_ok:
            print("Global position estimate OK")
            break

    # Arm the drone
    print("Arming drone...")
    await drone.action.arm()

    # Take off
    print("Taking off...")
    await drone.action.takeoff()

    # Wait for drone to reach takeoff altitude
    await asyncio.sleep(10)

    # Define mission parameters - adjust for 30x30 field
    # Assume origin (0, 0) is at the center of the field
    field_size = 15  # Half the field size in feet (30/2)
    mission_items = []

    # Example mission waypoints - adjust for your desired coverage pattern
    mission_items.append(MissionItem(47.398039859999997 + (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 1
    mission_items.append(MissionItem(47.398039859999997 - (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 2
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 + (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 3
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 - (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 4


    # Upload mission
    print("Uploading mission...")
    await drone.mission.upload_mission(mission_items)

    # Start mission
    print("Starting mission...")
    await drone.mission.start_mission()

    # ArUco marker detection loop - run while mission is active
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    aruco_params = cv2.aruco.DetectorParameters_create()

    while True:
        # Get camera frame - replace with your camera feed
        # frame = get_camera_frame()
        # ...
        frame = np.zeros((480,640,3),np.uint8) # Replace with actual frame from camera

        # Detect ArUco markers
        corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)

        if ids is not None:
            for i in range(len(ids)):
                # Get marker ID and position
                marker_id = ids[i][0]
                marker_position = corners[i][0].mean(axis=0)  # Average corner coordinates for center

                # Communicate with UGV
                communicate_with_ugv(marker_id, marker_position)

        # Check mission status - break loop if mission is complete
        mission_progress = await drone.mission.mission_progress()
        if mission_progress.current == mission_progress.total:
            break

        # Add a delay to avoid excessive processing
        await asyncio.sleep(0.1)

    # Return to launch
    print("Returning to launch...")
    await drone.action.return_to_launch()

    # Wait for return to launch to complete
    await asyncio.sleep(20)

    # Land
    print("Landing...")
    await drone.action.land()

async def main():
    try:
        await run()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Run the asyncio loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

SyntaxError: invalid syntax (<ipython-input-6-94f86b3250a6>, line 119)

In [7]:
#!/usr/bin/env python3

import asyncio
import cv2
import numpy as np
from mavsdk import System
from mavsdk.mission import MissionItem
# Assume 'communicate_with_ugv' is a function to send data to the UGV
# Replace with your actual communication implementation
def communicate_with_ugv(marker_id, marker_position):
    """Communicates the detected marker information to the UGV."""
    print(f"Detected marker ID: {marker_id}, Position: {marker_position}")
    # Replace with your UGV communication logic
    # ...

async def run():
    # Initialize the drone
    drone = System()

    # Connect to the drone
    print("Connecting to drone...")
    await drone.connect(system_address="udp://:14540")

    # Wait for connection
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print("Drone connected!")
            break

    # Check if vehicle is ready
    print("Waiting for drone to have a global position estimate...")
    async for health in drone.telemetry.health():
        if health.is_global_position_ok:
            print("Global position estimate OK")
            break

    # Arm the drone
    print("Arming drone...")
    await drone.action.arm()

    # Take off
    print("Taking off...")
    await drone.action.takeoff()

    # Wait for drone to reach takeoff altitude
    await asyncio.sleep(10)

    # Define mission parameters - adjust for 30x30 field
    # Assume origin (0, 0) is at the center of the field
    field_size = 15  # Half the field size in feet (30/2)
    mission_items = []

    # Example mission waypoints - adjust for your desired coverage pattern
    mission_items.append(MissionItem(47.398039859999997 + (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 1
    mission_items.append(MissionItem(47.398039859999997 - (field_size / 111139), 8.5455725400000002, 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 2
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 + (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 3
    mission_items.append(MissionItem(47.398039859999997, 8.5455725400000002 - (field_size / (111139 * np.cos(47.398039859999997))), 25, 2, True, float('nan'), float('nan'), MissionItem.CameraAction.NONE, float('nan'), float('nan'), 1.0, 0, 0)) # Waypoint 4


    # Upload mission
    print("Uploading mission...")
    await drone.mission.upload_mission(mission_items)

    # Start mission
    print("Starting mission...")
    await drone.mission.start_mission()

    # ArUco marker detection loop - run while mission is active
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    aruco_params = cv2.aruco.DetectorParameters_create()

    while True:
        # Get camera frame - replace with your camera feed
        # frame = get_camera_frame()
        # ...
        frame = np.zeros((480,640,3),np.uint8) # Replace with actual frame from camera

        # Detect ArUco markers
        corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)

        if ids is not None:
            for i in range(len(ids)):
                # Get marker ID and position
                marker_id = ids[i][0]
                marker_position = corners[i][0].mean(axis=0)  # Average corner coordinates for center

                # Communicate with UGV
                communicate_with_ugv(marker_id, marker_position)

        # Check mission status - break loop if mission is complete
        mission_progress = await drone.mission.mission_progress()
        if mission_progress.current == mission_progress.total:
            break

        # Add a delay to avoid excessive processing
        await asyncio.sleep(0.1)

    # Return to launch
    print("Returning to launch...")
    await drone.action.return_to_launch()

    # Wait for return to launch to complete
    await asyncio.sleep(20)

    # Land
    print("Landing...")
    await drone.action.land()

async def main():
    try:
        await run()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Run the asyncio loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

RuntimeError: This event loop is already running